# Proxy Variable - Age and Recidivism Comparison

The code bellow compares correlation and network output in an attempt to identitfy Proxy Varaibles.

NETWORK 1 - Trained without age variable

In [1]:
from conx import Network
import random

#seperate into arrays
#for loop on same index in each row
#Feed one network with race and other varaibles
#Feed another network with seriiousness of crime 

def getData(file):
    #open file
    tf = open(file, "r")
    content = tf.readlines()
    row_count = sum(1 for row in content)
    print(row_count)
    #Parse file
    input = []
    content = [x.strip() for x in content[1:]]
    for y in content:
        row = [col for col in y.split(",")]
        #print(row)
        input.append(row)
    tf.close()
    return input

cols = getData("compas-scores-two-years.csv")


7215


In [2]:
#In column 23 there was an issue with splitting the value stored at ","
#Resulting in a list containing 54 columns instead of 53
for c in cols:
    if len(c) == 54:
        c.remove(c[23])

In [ ]:
def parseInputs(inputs):
    new = []
    row = []
    countWhat = 0
    for r in inputs:
        if r[9] == "Caucasian" or r[9] == "African-American":
            #0
            if r[5] == "Male":
                row.append(1)
            elif r[5] == "Female":
                row.append(0)
            else:
                print("error parsing col 5, Error occurs on row " + str(countWhat))
            #race = Caucasian, African-American
            if r[9] == "Caucasian":
                row.append(0)
            elif r[9] == "African-American":
                row.append(1)
            else:
                print("error parsing col 9, Error occurs on row " + str(countWhat))
            #Risk of violence Score: Low, Medium, and High
            if r[40] == "Low":
                row.append(1)
                row.append(0)
                row.append(0)
            elif r[40] == "Medium":
                row.append(0)
                row.append(1)
                row.append(0)
            elif r[40] == "High":
                row.append(0)
                row.append(0)
                row.append(1)
            else:
                print("error parsing col 44, Error occurs on row " + str(countWhat))
            #Risk of recidi Score: Low, Medium, and High
            if r[44] == "Low":
                row.append(1)
                row.append(0)
                row.append(0)
            elif r[44] == "Medium":
                row.append(0)
                row.append(1)
                row.append(0)
            elif r[44] == "High":
                row.append(0)
                row.append(0)
                row.append(1)
            else:
                print("error parsing col 44, Error occurs on row " + str(countWhat))
            #If recidivism 
            if r[52] == '0':
                row.append(0)
            elif r[52] == '1':
                row.append(1)
            else:
                print("error parsing col 52, Error occurs on row " + str(countWhat))
            
            #parse error in row 4754, disposed of here
            if r[40] == "":
                r = []

            if row != [] and len(row) == 9:
                new.append(row)
            row = []
        countWhat += 1
    return new

tempinputs = parseInputs(cols)  

In [5]:
#Ensure inputs are coorect length
#Input how many rows to be processed
inputs = []
lengthInput = 4000
for r in range(lengthInput):
    inputs.append(tempinputs[r])  
    
inputs = [[x[0:8], x[8:]] for x in inputs]

#print inputs

In [7]:
net = Network(8, 60, 1)

#desired output/target
def des(inputs):
    re = inputs[9]
    return [int(re)]

In [8]:
inputs[0]

[[1, 1, 1, 0, 0, 1, 0, 0], [1]]

In [9]:
net.set_inputs(inputs)

In [10]:
net.reset()
#net.train(max_training_epochs=1000, momentum=1, epsilon=1)
net.train(max_training_epochs=1500, report_rate=250, tolerance = .4)

#stop_percentage: default 1.0, # used in train
#tolerance: default 0.1, # used in train
#report_rate: default 500, # used in train
#epsilon: default 0.1, # learning rate, used in train
#momentum: default 0.9, # used in train
#shuffle=True

--------------------------------------------------
Training for max trails: 1500 ...
Epoch: 0 TSS error: 2017.13141372 %correct: 0.46325
Epoch: 250 TSS error: 872.837582585 %correct: 0.50475
Epoch: 500 TSS error: 872.41954008 %correct: 0.49675
Epoch: 750 TSS error: 869.801563492 %correct: 0.50775
Epoch: 1000 TSS error: 869.184811526 %correct: 0.512
Epoch: 1250 TSS error: 870.876787912 %correct: 0.50725
Epoch: 1500 TSS error: 868.774355605 %correct: 0.51375
--------------------------------------------------
Epoch: 1500 TSS error: 868.774355605 %correct: 0.51375


In [11]:
net.test()

--------------------------------------------------
Test:
******************************
Input : [1, 0, 1, 0, 0, 1, 0, 0]
Output: [ 0.27992334] 
Target: [0] Correct
******************************
Input : [1, 1, 1, 0, 0, 1, 0, 0]
Output: [ 0.34722738] 
Target: [1] Incorrect
******************************
Input : [1, 1, 0, 0, 1, 0, 0, 1]
Output: [ 0.75109604] 
Target: [1] Correct
******************************
Input : [1, 1, 0, 1, 0, 0, 1, 0]
Output: [ 0.5223228] 
Target: [0] Incorrect
******************************
Input : [1, 0, 0, 1, 0, 1, 0, 0]
Output: [ 0.53327081] 
Target: [1] Incorrect
******************************
Input : [1, 1, 1, 0, 0, 1, 0, 0]
Output: [ 0.34722738] 
Target: [0] Correct
******************************
Input : [0, 0, 0, 1, 0, 1, 0, 0]
Output: [ 0.44364925] 
Target: [0] Incorrect
******************************
Input : [1, 0, 1, 0, 0, 1, 0, 0]
Output: [ 0.27992334] 
Target: [0] Correct
******************************
Input : [1, 1, 1, 0, 0, 1, 0, 0]
Output: [ 0.3472

In [13]:
#try network on test data
results = []
forit = len(tempinputs) - 4000
for h in range(forit):
    val = tempinputs[h + 4000]
    val = val[0:8]
    results.append(net.propagate(val))

NETWORK 2 - with recidi score

In [15]:
def parseInputs(inputs):
    new = []
    row = []
    countWhat = 0
    for r in inputs:
        if r[9] == "Caucasian" or r[9] == "African-American":
            #Age: Greater than 45, 25 - 45, and Less than 25
            #1, 2, and 3
            #0
            if r[5] == "Male":
                row.append(1)
            elif r[5] == "Female":
                row.append(0)
            else:
                print("error parsing col 5, Error occurs on row " + str(countWhat))
            #race = Caucasian, African-American
            if r[9] == "Caucasian":
                row.append(0)
            elif r[9] == "African-American":
                row.append(1)
            else:
                print("error parsing col 9, Error occurs on row " + str(countWhat))
            if r[8] == "Greater than 45":
                row.append(1)
                row.append(0)
                row.append(0)
            elif r[8] == "25 - 45":
                row.append(0)
                row.append(1)
                row.append(0)
            elif r[8] == "Less than 25":
                row.append(0)
                row.append(0)
                row.append(1)
            else:
                print("error parsing col 8, Error occurs on row " + str(countWhat))
            #Risk of violence Score: Low, Medium, and High
            if r[40] == "Low":
                row.append(1)
                row.append(0)
                row.append(0)
            elif r[40] == "Medium":
                row.append(0)
                row.append(1)
                row.append(0)
            elif r[40] == "High":
                row.append(0)
                row.append(0)
                row.append(1)
            else:
                print("error parsing col 44, Error occurs on row " + str(countWhat))
            #If recidivism 
            if r[52] == '0':
                row.append(0)
            elif r[52] == '1':
                row.append(1)
            else:
                print("error parsing col 52, Error occurs on row " + str(countWhat))
            
            #parse error in row 4754, disposed of here
            if r[40] == "":
                r = []

            if row != [] and len(row) == 9:
                new.append(row)
            row = []
        countWhat += 1
    return new

tempinputs = parseInputs(cols)  

error parsing col 44, Error occurs on row 4754


In [16]:
print(tempinputs[0])

[1, 1, 0, 1, 0, 1, 0, 0, 1]


In [17]:
print(cols[0])

['1', 'miguel hernandez', 'miguel', 'hernandez', '2013-08-14', 'Male', '1947-04-18', '69', 'Greater than 45', 'Other', '0', '1', '0', '0', '0', '-1', '2013-08-13 06:03:42', '2013-08-14 05:41:20', '13011352CF10A', '2013-08-13', '', '1', 'F', 'Aggravated Assault w/Firearm', '0', '', '', '', '', '', '', '', '', '0', '', '', '', '', 'Risk of Recidivism', '1', 'Low', '2013-08-14', 'Risk of Violence', '1', 'Low', '2013-08-14', '2014-07-07', '2014-07-14', '0', '0', '327', '0', '0']


In [18]:
#Reset inputs
inputs = []
print(tempinputs[0])
for r in range(lengthInput):
    inputs.append(tempinputs[r])  
    
inputs = [[x[0:8], x[8:]] for x in inputs]

[1, 1, 0, 1, 0, 1, 0, 0, 1]


In [19]:
inputs[0]

[[1, 1, 0, 1, 0, 1, 0, 0], [1]]

In [20]:
net2 = Network(8, 60, 1)

#desired output/target
def des(inputs):
    re = inputs[9]
    return [int(re)]

In [21]:
net2.set_inputs(inputs)

In [22]:
net2.reset()
#net.train(max_training_epochs=1000, momentum=1, epsilon=1)
net2.train(max_training_epochs=1500, report_rate=250, tolerance = .4)

--------------------------------------------------
Training for max trails: 1500 ...
Epoch: 0 TSS error: 1233.15696452 %correct: 0.28175
Epoch: 250 TSS error: 862.855339982 %correct: 0.49525
Epoch: 500 TSS error: 863.913584848 %correct: 0.511
Epoch: 750 TSS error: 864.647461914 %correct: 0.49875
Epoch: 1000 TSS error: 862.644638401 %correct: 0.49225
Epoch: 1250 TSS error: 860.978350396 %correct: 0.507
Epoch: 1500 TSS error: 863.60509592 %correct: 0.499
--------------------------------------------------
Epoch: 1500 TSS error: 863.60509592 %correct: 0.499


In [25]:
net2.test()

--------------------------------------------------
Test:
******************************
Input : [1, 0, 1, 0, 0, 1, 0, 0]
Output: [ 0.17192145] 
Target: [1] Incorrect
******************************
Input : [1, 1, 0, 1, 0, 0, 0, 1]
Output: [ 0.70419844] 
Target: [0] Incorrect
******************************
Input : [1, 1, 1, 0, 0, 0, 0, 1]
Output: [ 0.5461117] 
Target: [1] Incorrect
******************************
Input : [1, 1, 1, 0, 0, 1, 0, 0]
Output: [ 0.25368602] 
Target: [0] Correct
******************************
Input : [1, 1, 0, 0, 1, 0, 0, 1]
Output: [ 0.76777558] 
Target: [0] Incorrect
******************************
Input : [1, 1, 0, 0, 1, 0, 0, 1]
Output: [ 0.76777558] 
Target: [1] Correct
******************************
Input : [1, 0, 0, 0, 1, 0, 1, 0]
Output: [ 0.62232145] 
Target: [1] Correct
******************************
Input : [0, 0, 1, 0, 0, 1, 0, 0]
Output: [ 0.22797422] 
Target: [0] Correct
******************************
Input : [1, 1, 0, 1, 0, 1, 0, 0]
Output: [ 0.2908

Look at data range = len(tempinputs) - 4000

In [27]:
results2 = []
forIt = (len(tempinputs) - 4000)
for p in range(forIt):
    val = tempinputs[p + 4000]
    val = val[0:8]
    results2.append(net2.propagate(val))
    

In [28]:
print("Proxy Precentage: ")  

Proxy Precentage: 


In [29]:
countSame = 0
for s in range(len(results2)):
    if (results[s] > .5 and results2[s] > .5) or (results[s] < .5 and results2[s] < .5):
        countSame += 1
        
print(countSame)
print("Proxy Precentage: ")      
print(float(countSame)/float(len(results)))

1693
Proxy Precentage: 
0.7878082829222894


In [30]:
def getProxy(results, results2):
    countSame = 0
    for s in range(len(results2)):
        bothTrue = results[s] > .5 and results2[s] > .5
        bothFalse = results[s] < .5 and results2[s] < .5
        if bothTrue or bothFalse:
            countSame += 1     
    return (float(countSame)/float(len(results)))

print(getProxy(results, results2))

0.7878082829222894


Check against actual results for true/false?

In [ ]:
recic = 5, 6, 7

results2 = []
forIt = (len(tempinputs) - 4000)
for p in range(forIt):
    val = tempinputs[p + 4000]
    val1 = val[]
    val2 = val[]
    results2.append(net2.propagate(val))